In [1]:
import numpy as np
import pyomo.environ as pyo
from pyomo.environ import ConcreteModel, Var, ConstraintList, Objective, SolverFactory, value, RangeSet

import jax
import jax.numpy as jnp
from jax.experimental.ode import odeint

# -------------- helper libraries -------------- #
import sys
import os
import time
import pickle
import matplotlib.pyplot as plt
import pandas as pd

collocation2_path = os.path.abspath(os.path.join('..', 'utils'))

# Add the directory to sys.path
if collocation2_path not in sys.path:
    sys.path.append(collocation2_path)

from collocation import compute_weights, lagrange_derivative
from interpolation import BarycentricInterpolation
from data_generation import generate_ode_data
from non_parametric_collocation import collocate_data
from optimization_pyomo import ODEOptimizationModel as ODEOptimizationModel
from pickle_func import pickle_data, unpickle_data

# -------------- regular neural nets --------------
from neural_net import create_train_state, train, neural_ode

In [2]:
#---------------------------------------------DATA PARAMS---------------------------------------------#
N = 200
noise_level = 0.2
ode_type, params = "harmonic_oscillator", {"omega_squared": 2}
start_time, end_time = 0, 10
spacing_type = "chebyshev" # "equally_spaced" or "chebyshev"
initial_state = jnp.array([0.0, 1.0])

#--------------------------------------------GENERATE DATA--------------------------------------------#
t, y, y_noisy, true_derivatives = generate_ode_data(N, noise_level, ode_type, params, start_time, end_time, spacing_type, initial_state)

# numpy array is required for pyomo
y_noisy = np.array(jnp.squeeze(y_noisy))
t = np.array(jnp.squeeze(t))

#-----------------------------------------COLLOCATION MATRIX PREPARATION-------------------------------------------#
weights = compute_weights(t)
D = np.array(lagrange_derivative(t, weights))

#---------------------------------------------------TEST DATA--------------------------------------------#
t_test, y_test, _, _ = generate_ode_data(N*2, noise_level, ode_type, params, start_time, 20, "uniform", initial_state)

#--------------------------------------------NON-PARAMETRIC COLLOCATION--------------------------------------------#
estimated_derivative, estimated_solution = collocate_data(y_noisy, t, 'EpanechnikovKernel', bandwidth=0.5)

In [ ]:
layer_widths = [2, 20, 2]
results_tol = {}

tol_li = np.logspace(-4.0, -7.0, num=10)  

for tol in tol_li:
    #---------------------------------MODEL---------------------------------#
    ode_model = ODEOptimizationModel(y_noisy, t, D, layer_widths, y_init = estimated_solution, penalty_lambda = 0, tol = tol, verbose= True, print_level= 4)
    ode_model.build_model()
    result = ode_model.solve_model()
    
    #------------------------------ ODEINT PREP ----------------------------#
    def system_dynamics(y, t):
        u, v = y
        du_dt, dv_dt = ode_model.predict(u, v)
        return jnp.array([du_dt, dv_dt])
    
    initial_conditions = jnp.array([0.0, 1.0]) 
    #---------------------------MEAN ABSOLUTE ERROR--------------------------#

    y_pred_train = odeint(system_dynamics, initial_conditions, t)    
    mae_train = jnp.mean(jnp.abs(y_pred_train - y))
    
    y_pred_test = odeint(system_dynamics, initial_conditions, t_test)   
    mae_test = jnp.mean(jnp.abs(y_pred_test[200:] - y_test[200:]))
    
    #------------------------------SAVING RESULTS----------------------------#

    results_tol[tol] = {
        "tol":tol, "y_pred_train": y_pred_train, "y_pred_test" : y_pred_test, "t" : t,
        "time_elapsed": result['solver_time'], "mae_test" : mae_test, "mae":mae_train,
        "termination_condition": result['termination_condition'], "message": result['message']
        }